# FESDModel

FESD - Fault estimation for skeleton detection - is a suite that aims at finding faults in joints of skeletons, which are detected by human pose estimatiors.

FESDData is the sister project to this notebook, which aims at recording depth and rgb data, as well as populating the data with human poses from variing human pose estimators.

Furthermore, FESTData augments all data based on joint confidence.

FFESDModel aims to develop and evaluate a model based on the faulty and augmented joint data as well as RGBD data.

## Libraries

We need a range of libraries which are imported here. We also define some constants.

In [1]:
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tqdm

import cv2
import pyrealsense2 as rs

sns.set_style()

In [2]:
RECORDING_DIR = '/home/leo/Recordings/'
EXERCISES_FILE_NAME = 'Exercises.json'

## Data Loading

Firstly we need to import all the recordings into the notebook.


### Load Metadata

It is important to load the metadata, such as the session parameters, the exercises and the recording paths.

In [3]:
recording_jsons = []
for file in os.listdir(RECORDING_DIR):
  if (file.endswith('.json') and 
      not file.endswith('Skeleton.json') and 
      not file.endswith(EXERCISES_FILE_NAME)):
    with open(file=os.path.join(RECORDING_DIR, file), mode='r') as file:
      data = json.load(file)
      recording_jsons.append(data)

len(recording_jsons)

8

Then we load the Exercises from the exercise file.

In [4]:
exercises_json = []

with open(file=os.path.join(RECORDING_DIR, EXERCISES_FILE_NAME), mode='r') as file:
  exercises_json = json.load(file)['Exercises']

len(exercises_json)

13

Next we extract all recordings from the sessions.

In [5]:
recording_paths_rs = []
recording_paths_orbbec = []

for recording in recording_jsons:
  for camera in recording['Cameras']:
    if camera['Type'] == 'Realsense':
      recording_paths_rs.append(camera['FileName'])

len(recording_paths_rs)

8

### Load Frames from Realsense Camera

Next we load the frames from the realsense recordings into arrays of CV matrixes.


In [14]:
def load_recording_rs(recording_path_rs, num_frames: int):
  pipeline = rs.pipeline()
  config = rs.config()

  rs.config.enable_device_from_file(config, os.path.join(RECORDING_DIR, recording_path_rs))
  


  # Configure the pipeline to stream the depth stream
  # Change this parameters according to the recorded bag file resolution
  config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
  config.enable_stream(rs.stream.color, 1280, 720, rs.format.rgb8, 30)

  # Start streaming from file
  pipeline.start(config)
  device = pipeline.get_active_profile().get_device()
  
  device.as_playback().set_real_time(False)
    
  depth_scale = device.first_depth_sensor().get_depth_scale()
  
  # Create colorizer object
  cv_depth_frames = []
  cv_color_frames = []
  
  align = rs.align(rs.stream.depth)

  # Streaming loop
  for i in tqdm.tqdm(range(num_frames)):

    # Get frameset of depth
    frames = pipeline.wait_for_frames()
    frames = align.process(frames)
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()
    if not depth_frame or not color_frame:
      continue    

    color_frame = np.asanyarray(color_frame.get_data())
    color_frame = cv2.cvtColor(color_frame, cv2.COLOR_BGR2RGB)
    #color_frame = cv2.resize(color_frame, dsize=(640, 480), interpolation=cv2.INTER_AREA)    

    # Get depth frame
    cv_depth_frames.append(np.asanyarray(depth_frame.get_data()) * depth_scale)
    cv_color_frames.append(color_frame)
  
  pipeline.stop()

  return cv_depth_frames, cv_color_frames

In [15]:
cv_depth_videos = []
cv_color_videos = []

for (i, recording_path_rs) in enumerate(recording_paths_rs):
  print(f'Loading recording {i+1} of {len(recording_paths_rs)} for recording {recording_jsons[i]["Name"]}')
  depth, color = load_recording_rs(recording_path_rs, recording_jsons[i]['Frames'])
  cv_depth_videos.append(depth)
  cv_color_videos.append(color)
  

Loading recording 1 of 8 for recording Session 2023-02-08T22:23:46


100%|██████████| 602/602 [00:02<00:00, 202.41it/s]


Loading recording 2 of 8 for recording Session 2023-02-08T22:28:08


100%|██████████| 602/602 [00:03<00:00, 178.77it/s]


Loading recording 3 of 8 for recording Session 2023-02-08T22:22:59


...

100%|██████████| 599/599 [00:02<00:00, 207.97it/s]


Loading recording 7 of 8 for recording Session 2023-02-08T22:26:37


100%|██████████| 601/601 [00:03<00:00, 195.74it/s]


Loading recording 8 of 8 for recording Session 2023-02-08T22:25:45


100%|██████████| 601/601 [00:02<00:00, 211.49it/s]


In [20]:
cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)

num_sample = 300

for (i, recording_path_rs) in enumerate(recording_paths_rs):
    for j in range(num_sample):
        depth_image = cv_depth_videos[i][j]
        color_image = cv_color_videos[i][j]

        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=255/8), cv2.COLORMAP_INFERNO)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        added_image = cv2.addWeighted(depth_colormap,0.4,color_image,0.1,0)
        
        images = np.hstack((color_image, depth_colormap, added_image))

        # Show images
        cv2.imshow('RealSense', images)
        cv2.waitKey(1)
        
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x56164e958dc0) is not the object's thread (0x56160ca6b490).
Cannot move to target thread (0x56164e958dc0)

QObject::moveToThread: Current thread (0x56164e958dc0) is not the object's thread (0x56160ca6b490).
Cannot move to target thread (0x56164e958dc0)

QObject::moveToThread: Current thread (0x56164e958dc0) is not the object's thread (0x56160ca6b490).
Cannot move to target thread (0x56164e958dc0)

QObject::moveToThread: Current thread (0x56164e958dc0) is not the object's thread (0x56160ca6b490).
Cannot move to target thread (0x56164e958dc0)

QObject::moveToThread: Current thread (0x56164e958dc0) is not the object's thread (0x56160ca6b490).
Cannot move to target thread (0x56164e958dc0)

QObject::moveToThread: Current thread (0x56164e958dc0) is not the object's thread (0x56160ca6b490).
Cannot move to target thread (0x56164e958dc0)

QObject::moveToThread: Current thread (0x56164e958dc0) is not the object's thread (0x56160ca6b490).
Cannot move to tar

### Load Skeleton data

Next we load the human pose estimation data. The human pose estimation data is stored in a json.